In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict ={n:i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict)

In [7]:
seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']


In [90]:
x, y = [], []

for seq in seq_data:
    input = [word_dict[n] for n in seq[-1]]
    target = word_dict[seq[-1]]
    x.append(np.eye(n_class)[input])
    y.append(target)

In [97]:
i = [word_dict[n] for n in 'mak']
s = np.eye(n_class)[i]
s = torch.FloatTensor(s)
print(s.shape)
print(s)

torch.Size([3, 26])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


In [92]:
x = torch.FloatTensor(x)
y = torch.LongTensor(y)

In [18]:
x.shape

torch.Size([10, 1, 26])

In [23]:
n_hidden = 128
lstm = nn.LSTM(input_size = n_class, hidden_size = n_hidden)
w = nn.Linear(n_hidden, n_class, bias = True)


In [101]:
def make_batch():
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

input_batch, target_batch = make_batch()
input_batch = torch.FloatTensor(input_batch)
print(input_batch.shape)

torch.Size([10, 3, 26])


In [102]:
input = input_batch.transpose(0,1)

In [103]:
input.shape

torch.Size([3, 10, 26])

In [104]:
input

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [106]:
h_s = torch.zeros(1, len(x), n_hidden)
h_s

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [108]:
c_s = torch.zeros(1, len(x), n_hidden)
c_s

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [112]:
outputs = lstm(input)

In [113]:
outputs

(tensor([[[-3.6840e-03, -7.5072e-03, -2.3415e-02,  ...,  7.3702e-03,
           -7.0652e-03,  2.0072e-02],
          [ 1.3210e-02,  2.7112e-02, -3.4137e-02,  ...,  1.5673e-02,
           -2.7442e-02,  1.1476e-02],
          [-6.1603e-04,  2.4725e-02, -3.3240e-02,  ..., -7.5037e-03,
           -1.2266e-04,  9.9289e-03],
          ...,
          [-1.7928e-02, -1.0447e-02, -7.9844e-05,  ..., -2.1821e-02,
            1.0516e-02, -1.1600e-02],
          [-1.7928e-02, -1.0447e-02, -7.9844e-05,  ..., -2.1821e-02,
            1.0516e-02, -1.1600e-02],
          [-1.2016e-02, -9.5797e-03, -2.5770e-02,  ...,  4.4860e-03,
           -1.9494e-03,  2.1715e-02]],
 
         [[-5.5865e-03,  1.9197e-02, -3.2526e-02,  ..., -2.2193e-03,
            6.8906e-03,  5.5899e-03],
          [ 7.5826e-03,  1.8998e-02, -5.8732e-02,  ...,  2.2173e-02,
           -3.3730e-02,  3.1290e-02],
          [ 2.8546e-03,  1.5464e-02, -2.4864e-02,  ..., -1.8692e-02,
           -2.2926e-02,  1.2291e-02],
          ...,
    

In [120]:
o, (_,_) = lstm(input, (h_s, c_s))

In [124]:
o1 = o[-1]
y_1 = w(o1)
print(y_1.shape)

torch.Size([10, 26])


In [121]:
y_hat = w(o)

In [123]:
y_hat.shape

torch.Size([3, 10, 26])

In [129]:
y_hat

tensor([[[ 5.4160e-02,  4.8706e-02, -2.1634e-02, -1.4525e-03,  5.8699e-05,
           6.2127e-02,  9.1155e-03,  5.7406e-03,  3.6438e-02, -4.3025e-02,
          -2.5260e-03, -7.9446e-02, -8.2064e-02, -4.9997e-02,  4.1076e-03,
           7.6286e-02, -5.7395e-02, -4.0604e-03, -5.6325e-02, -5.1519e-02,
          -4.4776e-02,  8.5573e-02,  5.6333e-02,  1.1348e-03,  6.1539e-02,
          -1.6350e-02],
         [ 3.7802e-02,  4.9323e-02, -1.9219e-02, -6.9809e-03,  8.5908e-03,
           6.5433e-02,  7.5247e-03, -1.1907e-03,  4.1599e-02, -7.0214e-02,
           6.6599e-04, -7.3182e-02, -9.1326e-02, -4.0560e-02, -1.8847e-02,
           6.8153e-02, -6.0746e-02,  5.9341e-03, -4.7345e-02, -3.8669e-02,
          -5.8229e-02,  9.0159e-02,  6.7963e-02, -1.0165e-02,  6.4469e-02,
          -1.0020e-02],
         [ 4.9266e-02,  4.5075e-02, -3.2685e-02, -5.0520e-03,  2.5581e-03,
           5.7628e-02,  1.8103e-02,  2.0666e-03,  3.2074e-02, -5.1402e-02,
          -1.0251e-02, -5.9089e-02, -9.0004e-02, -4.

In [132]:
c = y_hat.view(-1,78)
c

tensor([[ 5.4160e-02,  4.8706e-02, -2.1634e-02, -1.4525e-03,  5.8699e-05,
          6.2127e-02,  9.1155e-03,  5.7406e-03,  3.6438e-02, -4.3025e-02,
         -2.5260e-03, -7.9446e-02, -8.2064e-02, -4.9997e-02,  4.1076e-03,
          7.6286e-02, -5.7395e-02, -4.0604e-03, -5.6325e-02, -5.1519e-02,
         -4.4776e-02,  8.5573e-02,  5.6333e-02,  1.1348e-03,  6.1539e-02,
         -1.6350e-02,  3.7802e-02,  4.9323e-02, -1.9219e-02, -6.9809e-03,
          8.5908e-03,  6.5433e-02,  7.5247e-03, -1.1907e-03,  4.1599e-02,
         -7.0214e-02,  6.6599e-04, -7.3182e-02, -9.1326e-02, -4.0560e-02,
         -1.8847e-02,  6.8153e-02, -6.0746e-02,  5.9341e-03, -4.7345e-02,
         -3.8669e-02, -5.8229e-02,  9.0159e-02,  6.7963e-02, -1.0165e-02,
          6.4469e-02, -1.0020e-02,  4.9266e-02,  4.5075e-02, -3.2685e-02,
         -5.0520e-03,  2.5581e-03,  5.7628e-02,  1.8103e-02,  2.0666e-03,
          3.2074e-02, -5.1402e-02, -1.0251e-02, -5.9089e-02, -9.0004e-02,
         -4.0496e-02, -8.2096e-07,  5.

In [118]:
y

tensor([ 4,  3, 11,  3,  4,  4,  4,  4,  7, 17])

In [119]:
criterion = nn.CrossEntropyLoss()
c = criterion(y_hat, y)
print(c)


tensor(3.2628, grad_fn=<NllLossBackward>)
